# Load Library

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings 
warnings.filterwarnings('ignore')

# Load Data

In [2]:
data_path = 'C:/Users/USER/DACON/creditcard/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
submission = pd.read_csv(data_path+'sample_submission.csv')

# Data Preprocessing

## (1) Deleting

### index

In [3]:
train = train.drop('index', axis=1, inplace=False)
test = test.drop('index', axis=1, inplace=False)

### FLAG_MOBIL 

In [4]:
train = train.drop('FLAG_MOBIL', axis=1, inplace=False)
test = test.drop('FLAG_MOBIL', axis=1, inplace=False)

### occyp_type
* 결측치가 많으므로 제거
* 단, 결측치를 하나의 범주를 보는 등 다른 처리도 고려해볼만 함 

In [5]:
# # 결측치 제거
# train = train.drop('occyp_type', axis=1, inplace=False)
# test = test.drop('occyp_type', axis=1, inplace=False)

In [6]:
# 결측치 대체
train['occyp_type'] = train['occyp_type'].fillna('inoccupation')
test['occyp_type'] = test['occyp_type'].fillna('inoccupation')

## (2) Encoding

### Binary

In [7]:
train['gender'] = train['gender'].replace(['F','M'],[0,1])
test['gender'] = test['gender'].replace(['F','M'],[0,1])
train['car'] = train['car'].replace(['N','Y'],[0,1])
test['car'] = test['car'].replace(['N','Y'],[0,1])
train['reality'] = train['reality'].replace(['N','Y'],[0,1])
test['reality'] = test['reality'].replace(['N','Y'],[0,1])

### Ordinal

In [8]:
# edu_order = {
#     'Lower secondary' : 0, # 초딩
#     'Secondary / secondary special' : 1, # 중딩 
#     'Incomplete higher' : 2, # 자퇴
#     'Higher education' : 3, # 고딩
#      'Academic degree' : 4 # 대딩
# }

# train.edu_type = train.edu_type.map(edu_order)
# test.edu_type = test.edu_type.map(edu_order)

### * Categorical

In [9]:
# # LabelEncoding
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()

# train['income_type'] = encoder.fit_transform(train['income_type'])
# test['income_type'] = encoder.transform(test['income_type'])
# train['family_type'] = encoder.fit_transform(train['family_type'])
# test['family_type'] = encoder.transform(test['family_type'])
# train['house_type'] = encoder.fit_transform(train['house_type'])
# test['house_type'] = encoder.transform(test['house_type'])

In [10]:
# OneHotEncoding
train = pd.get_dummies(train)
test = pd.get_dummies(test)

## (3) Scaling

### family_size & child_num
* 두 변수의 상관관계가 높은 것을 보아, 다중공선성이 의심되므로 둘 중 하나를 제거
* 극단적인 값이 포함된 것 같으므로 임의로 조정하거나 이상치 처리가 필요할 듯

In [11]:
# # family_size 제거
# train = train.drop('family_size', axis=1, inplace=False)
# test = test.drop('family_size', axis=1, inplace=False)

# # child_num이 2 이상인 경우 처리
# train.loc[train['child_num'] >= 7,'child_num'] = 7
# test.loc[test['child_num'] >= 7,'child_num'] = 7

In [12]:
# # child_num 제거
# train = train.drop('child_num', axis=1, inplace=False)
# test = test.drop('child_num', axis=1, inplace=False)

# # family_size가 5 이상인 경우 처리
# train.loc[train['family_size'] >= 14,'family_size'] = 14
# test.loc[test['family_size'] >= 14,'family_size'] = 14

### income_total

In [13]:
# train['income_total'] = train['income_total'].apply(np.log)
# test['income_total'] = test['income_total'].apply(np.log)

### DAYS_BIRTH

In [14]:
def days_to_age(x): 
    return (x*-1)/30
train['DAYS_BIRTH'] = train['DAYS_BIRTH'].apply(days_to_age)
train = train.astype({'DAYS_BIRTH': 'int'})
test['DAYS_BIRTH'] = test['DAYS_BIRTH'].apply(days_to_age)
test = test.astype({'DAYS_BIRTH': 'int'})

In [15]:
# # 제거
# train = train.drop('DAYS_BIRTH', axis=1, inplace=False)
# test = test.drop('DAYS_BIRTH', axis=1, inplace=False)

### DAYS_EMPLOYED

In [16]:
train.loc[(train.DAYS_EMPLOYED)>=0,'DAYS_EMPLOYED'] = 0 
train['DAYS_EMPLOYED'] = train['DAYS_EMPLOYED'].apply(days_to_age)
train = train.astype({'DAYS_EMPLOYED': 'int'})
test.loc[(test.DAYS_EMPLOYED)>=0,'DAYS_EMPLOYED'] = 0 
test['DAYS_EMPLOYED'] = test['DAYS_EMPLOYED'].apply(days_to_age)
test = test.astype({'DAYS_EMPLOYED': 'int'})

In [17]:
# # 제거
# train = train.drop('DAYS_EMPLOYED', axis=1, inplace=False)
# test = test.drop('DAYS_EMPLOYED', axis=1, inplace=False)

### begin_month

In [18]:
def minus(x):
    return x * -1
train['begin_month'] = train['begin_month'].apply(minus)
train = train.astype({'begin_month': 'int'})
test['begin_month'] = test['begin_month'].apply(minus)
test = test.astype({'begin_month': 'int'})

In [19]:
# # 제거
# train = train.drop('begin_month', axis=1, inplace=False)
# test = test.drop('begin_month', axis=1, inplace=False)

### 표준화

In [20]:
# StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train = train.drop('credit', axis=1)
scaled_train = scaler.fit_transform(train)
scaled_test = scaler.transform(test)
df_scaled_train = pd.DataFrame(data=scaled_train, columns=train.columns) 
df_scaled_test = pd.DataFrame(data=scaled_test, columns=test.columns) 

### 정규화

In [21]:
# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler()
scaled_train = scaler2.fit_transform(df_scaled_train)
scaled_test = scaler2.transform(df_scaled_test)
df_scaled_train = pd.DataFrame(data=scaled_train, columns=train.columns) 
df_scaled_test = pd.DataFrame(data=scaled_test, columns=test.columns) 

## (4) Class Imbalance
* https://dining-developer.tistory.com/27
* 단순 오버샘플링으로 해결하면 오버피팅 문제 발생
* 알고리즘을 통한 오버샘플링(ex. SMOTE, ADASYN) 찾아보기

### X/y 분리

In [22]:
# # 스케일링 X
# X = train.drop('credit', axis=1)
# y = train[['credit']]
# test = test 

In [23]:
# 스케일링 O
train = pd.read_csv(data_path+'train.csv')
test = df_scaled_test 
X = df_scaled_train
y = train[['credit']]

### RandomOverSampling

In [24]:
# from imblearn.over_sampling import RandomOverSampler
# X, y = RandomOverSampler().fit_sample(X, y)

### SMOTE

In [25]:
# from imblearn.over_sampling import SMOTE
# X, y = SMOTE().fit_resample(X, y)

### train/val 분리

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

# Modeling
* log loss를 줄이는 모형 적합 방향: https://dacon.io/competitions/open/235551/talkboard/400298
* 오버피팅 극복: https://nittaku.tistory.com/289

## (1) LightGBM

In [30]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss

In [33]:
# # GridSearchCV
# lgbm = LGBMClassifier(n_classifier=100)
# params = {'max_depth':[128,160],'num_leaves':[32,64],'min_child_samples':[60,100],'subsample':[0.8,1]}
# grid = GridSearchCV(lgbm, param_grid=params, cv=5)
# grid.fit(X_train, y_train, early_stopping_rounds=30, eval_metric='logloss', eval_set=[(X_train,y_train),(X_val,y_val)])
# print('GridSearchCV 최적 파라미터:', grid.best_params_)

[1]	valid_0's multi_logloss: 0.86777	valid_1's multi_logloss: 0.868934
Training until validation scores don't improve for 30 rounds
[2]	valid_0's multi_logloss: 0.85612	valid_1's multi_logloss: 0.858284
[3]	valid_0's multi_logloss: 0.846513	valid_1's multi_logloss: 0.849945
[4]	valid_0's multi_logloss: 0.83862	valid_1's multi_logloss: 0.843182
[5]	valid_0's multi_logloss: 0.831666	valid_1's multi_logloss: 0.837256
[6]	valid_0's multi_logloss: 0.825338	valid_1's multi_logloss: 0.832075
[7]	valid_0's multi_logloss: 0.819782	valid_1's multi_logloss: 0.82753
[8]	valid_0's multi_logloss: 0.815103	valid_1's multi_logloss: 0.823888
[9]	valid_0's multi_logloss: 0.810548	valid_1's multi_logloss: 0.820687
[10]	valid_0's multi_logloss: 0.806361	valid_1's multi_logloss: 0.817604
[11]	valid_0's multi_logloss: 0.802552	valid_1's multi_logloss: 0.814922
[12]	valid_0's multi_logloss: 0.799055	valid_1's multi_logloss: 0.81236
[13]	valid_0's multi_logloss: 0.795909	valid_1's multi_logloss: 0.810234
[14]

[16]	valid_0's multi_logloss: 0.788941	valid_1's multi_logloss: 0.806316
[17]	valid_0's multi_logloss: 0.786522	valid_1's multi_logloss: 0.804914
[18]	valid_0's multi_logloss: 0.784348	valid_1's multi_logloss: 0.803712
[19]	valid_0's multi_logloss: 0.781969	valid_1's multi_logloss: 0.80237
[20]	valid_0's multi_logloss: 0.779849	valid_1's multi_logloss: 0.801326
[21]	valid_0's multi_logloss: 0.777864	valid_1's multi_logloss: 0.800277
[22]	valid_0's multi_logloss: 0.776024	valid_1's multi_logloss: 0.799433
[23]	valid_0's multi_logloss: 0.774367	valid_1's multi_logloss: 0.798834
[24]	valid_0's multi_logloss: 0.772304	valid_1's multi_logloss: 0.797531
[25]	valid_0's multi_logloss: 0.770567	valid_1's multi_logloss: 0.796599
[26]	valid_0's multi_logloss: 0.768902	valid_1's multi_logloss: 0.795893
[27]	valid_0's multi_logloss: 0.767344	valid_1's multi_logloss: 0.795057
[28]	valid_0's multi_logloss: 0.76565	valid_1's multi_logloss: 0.794267
[29]	valid_0's multi_logloss: 0.764296	valid_1's mult

[33]	valid_0's multi_logloss: 0.756598	valid_1's multi_logloss: 0.791826
[34]	valid_0's multi_logloss: 0.755388	valid_1's multi_logloss: 0.791369
[35]	valid_0's multi_logloss: 0.754021	valid_1's multi_logloss: 0.790875
[36]	valid_0's multi_logloss: 0.752809	valid_1's multi_logloss: 0.790464
[37]	valid_0's multi_logloss: 0.75158	valid_1's multi_logloss: 0.790119
[38]	valid_0's multi_logloss: 0.750367	valid_1's multi_logloss: 0.789773
[39]	valid_0's multi_logloss: 0.749063	valid_1's multi_logloss: 0.789387
[40]	valid_0's multi_logloss: 0.747813	valid_1's multi_logloss: 0.789066
[41]	valid_0's multi_logloss: 0.746804	valid_1's multi_logloss: 0.788849
[42]	valid_0's multi_logloss: 0.745567	valid_1's multi_logloss: 0.788327
[43]	valid_0's multi_logloss: 0.744313	valid_1's multi_logloss: 0.787944
[44]	valid_0's multi_logloss: 0.743268	valid_1's multi_logloss: 0.787565
[45]	valid_0's multi_logloss: 0.742099	valid_1's multi_logloss: 0.787039
[46]	valid_0's multi_logloss: 0.74059	valid_1's mult

[46]	valid_0's multi_logloss: 0.741561	valid_1's multi_logloss: 0.786965
[47]	valid_0's multi_logloss: 0.740217	valid_1's multi_logloss: 0.786497
[48]	valid_0's multi_logloss: 0.739175	valid_1's multi_logloss: 0.786435
[49]	valid_0's multi_logloss: 0.738016	valid_1's multi_logloss: 0.7862
[50]	valid_0's multi_logloss: 0.736773	valid_1's multi_logloss: 0.78573
[51]	valid_0's multi_logloss: 0.735428	valid_1's multi_logloss: 0.785371
[52]	valid_0's multi_logloss: 0.734072	valid_1's multi_logloss: 0.784815
[53]	valid_0's multi_logloss: 0.733017	valid_1's multi_logloss: 0.784376
[54]	valid_0's multi_logloss: 0.731732	valid_1's multi_logloss: 0.783944
[55]	valid_0's multi_logloss: 0.730652	valid_1's multi_logloss: 0.783741
[56]	valid_0's multi_logloss: 0.729527	valid_1's multi_logloss: 0.783452
[57]	valid_0's multi_logloss: 0.728514	valid_1's multi_logloss: 0.783084
[58]	valid_0's multi_logloss: 0.727515	valid_1's multi_logloss: 0.782819
[59]	valid_0's multi_logloss: 0.726317	valid_1's multi

[62]	valid_0's multi_logloss: 0.724911	valid_1's multi_logloss: 0.780733
[63]	valid_0's multi_logloss: 0.724034	valid_1's multi_logloss: 0.780563
[64]	valid_0's multi_logloss: 0.723245	valid_1's multi_logloss: 0.780292
[65]	valid_0's multi_logloss: 0.722302	valid_1's multi_logloss: 0.780121
[66]	valid_0's multi_logloss: 0.721395	valid_1's multi_logloss: 0.779946
[67]	valid_0's multi_logloss: 0.720343	valid_1's multi_logloss: 0.779821
[68]	valid_0's multi_logloss: 0.719396	valid_1's multi_logloss: 0.779662
[69]	valid_0's multi_logloss: 0.718348	valid_1's multi_logloss: 0.77924
[70]	valid_0's multi_logloss: 0.71751	valid_1's multi_logloss: 0.779043
[71]	valid_0's multi_logloss: 0.716677	valid_1's multi_logloss: 0.778763
[72]	valid_0's multi_logloss: 0.715747	valid_1's multi_logloss: 0.778589
[73]	valid_0's multi_logloss: 0.71466	valid_1's multi_logloss: 0.778059
[74]	valid_0's multi_logloss: 0.713977	valid_1's multi_logloss: 0.778022
[75]	valid_0's multi_logloss: 0.713065	valid_1's multi

...

[7]	valid_0's multi_logloss: 0.81002	valid_1's multi_logloss: 0.824277
[8]	valid_0's multi_logloss: 0.803993	valid_1's multi_logloss: 0.819965
[9]	valid_0's multi_logloss: 0.798511	valid_1's multi_logloss: 0.816356
[10]	valid_0's multi_logloss: 0.793307	valid_1's multi_logloss: 0.81328
[11]	valid_0's multi_logloss: 0.788345	valid_1's multi_logloss: 0.810476
[12]	valid_0's multi_logloss: 0.783998	valid_1's multi_logloss: 0.807947
[13]	valid_0's multi_logloss: 0.779653	valid_1's multi_logloss: 0.805233
[14]	valid_0's multi_logloss: 0.775643	valid_1's multi_logloss: 0.803182
[15]	valid_0's multi_logloss: 0.771809	valid_1's multi_logloss: 0.801471
[16]	valid_0's multi_logloss: 0.768216	valid_1's multi_logloss: 0.799283
[17]	valid_0's multi_logloss: 0.764483	valid_1's multi_logloss: 0.797329
[18]	valid_0's multi_logloss: 0.76105	valid_1's multi_logloss: 0.795629
[19]	valid_0's multi_logloss: 0.758146	valid_1's multi_logloss: 0.794159
[20]	valid_0's multi_logloss: 0.755035	valid_1's multi_lo

[21]	valid_0's multi_logloss: 0.753198	valid_1's multi_logloss: 0.7928
[22]	valid_0's multi_logloss: 0.750266	valid_1's multi_logloss: 0.791616
[23]	valid_0's multi_logloss: 0.74744	valid_1's multi_logloss: 0.790647
[24]	valid_0's multi_logloss: 0.744745	valid_1's multi_logloss: 0.789485
[25]	valid_0's multi_logloss: 0.742246	valid_1's multi_logloss: 0.788716
[26]	valid_0's multi_logloss: 0.739837	valid_1's multi_logloss: 0.787864
[27]	valid_0's multi_logloss: 0.737378	valid_1's multi_logloss: 0.786901
[28]	valid_0's multi_logloss: 0.734885	valid_1's multi_logloss: 0.786008
[29]	valid_0's multi_logloss: 0.732406	valid_1's multi_logloss: 0.784966
[30]	valid_0's multi_logloss: 0.729989	valid_1's multi_logloss: 0.784354
[31]	valid_0's multi_logloss: 0.727768	valid_1's multi_logloss: 0.783602
[32]	valid_0's multi_logloss: 0.725448	valid_1's multi_logloss: 0.782996
[33]	valid_0's multi_logloss: 0.723446	valid_1's multi_logloss: 0.782148
[34]	valid_0's multi_logloss: 0.721359	valid_1's multi

[33]	valid_0's multi_logloss: 0.725758	valid_1's multi_logloss: 0.784622
[34]	valid_0's multi_logloss: 0.723759	valid_1's multi_logloss: 0.783908
[35]	valid_0's multi_logloss: 0.721916	valid_1's multi_logloss: 0.783511
[36]	valid_0's multi_logloss: 0.72015	valid_1's multi_logloss: 0.782987
[37]	valid_0's multi_logloss: 0.71826	valid_1's multi_logloss: 0.782425
[38]	valid_0's multi_logloss: 0.716211	valid_1's multi_logloss: 0.781793
[39]	valid_0's multi_logloss: 0.714427	valid_1's multi_logloss: 0.781384
[40]	valid_0's multi_logloss: 0.712583	valid_1's multi_logloss: 0.781134
[41]	valid_0's multi_logloss: 0.71077	valid_1's multi_logloss: 0.780584
[42]	valid_0's multi_logloss: 0.709066	valid_1's multi_logloss: 0.780419
[43]	valid_0's multi_logloss: 0.707353	valid_1's multi_logloss: 0.780288
[44]	valid_0's multi_logloss: 0.70552	valid_1's multi_logloss: 0.779888
[45]	valid_0's multi_logloss: 0.703746	valid_1's multi_logloss: 0.77938
[46]	valid_0's multi_logloss: 0.702028	valid_1's multi_l

[48]	valid_0's multi_logloss: 0.681867	valid_1's multi_logloss: 0.76906
[49]	valid_0's multi_logloss: 0.679954	valid_1's multi_logloss: 0.768432
[50]	valid_0's multi_logloss: 0.678197	valid_1's multi_logloss: 0.768185
[51]	valid_0's multi_logloss: 0.676288	valid_1's multi_logloss: 0.767582
[52]	valid_0's multi_logloss: 0.674263	valid_1's multi_logloss: 0.76722
[53]	valid_0's multi_logloss: 0.672622	valid_1's multi_logloss: 0.766983
[54]	valid_0's multi_logloss: 0.670776	valid_1's multi_logloss: 0.766559
[55]	valid_0's multi_logloss: 0.668897	valid_1's multi_logloss: 0.765915
[56]	valid_0's multi_logloss: 0.667071	valid_1's multi_logloss: 0.765683
[57]	valid_0's multi_logloss: 0.665535	valid_1's multi_logloss: 0.765462
[58]	valid_0's multi_logloss: 0.663695	valid_1's multi_logloss: 0.765129
[59]	valid_0's multi_logloss: 0.661985	valid_1's multi_logloss: 0.764939
[60]	valid_0's multi_logloss: 0.660239	valid_1's multi_logloss: 0.764321
[61]	valid_0's multi_logloss: 0.658497	valid_1's mult

GridSearchCV(cv=5, estimator=LGBMClassifier(n_classifier=100),
             param_grid={'max_depth': [128, 160],
                         'min_child_samples': [60, 100], 'num_leaves': [32, 64],
                         'subsample': [0.8, 1]})

In [36]:
# StratifiedKFold
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

outcomes = []
sub = np.zeros((test.shape[0],3))
for n_fold, (train_index, val_index)  in enumerate(folds.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    lgb = LGBMClassifier(n_estimators=1000, num_leaves=64, sumsample=0.8,
                         min_child_samples=60, max_depth=128, random_state=2)
    lgb.fit(X_train, y_train,
           eval_set=[(X_train,y_train),(X_val,y_val)],
           early_stopping_rounds=30,
           verbose=100)
    predictions = lgb.predict_proba(X_val)
    y_val_onehot = pd.get_dummies(y_val)

    sub += lgb.predict_proba(test)

    logloss = log_loss(y_val_onehot, predictions)
    outcomes.append(logloss)

sub = sub / 10
   
np.mean(outcomes)

Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.617628	valid_1's multi_logloss: 0.730414
[200]	training's multi_logloss: 0.523076	valid_1's multi_logloss: 0.718041
Early stopping, best iteration is:
[230]	training's multi_logloss: 0.500074	valid_1's multi_logloss: 0.716081
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.613321	valid_1's multi_logloss: 0.747676
[200]	training's multi_logloss: 0.520062	valid_1's multi_logloss: 0.735162
[300]	training's multi_logloss: 0.45492	valid_1's multi_logloss: 0.732506
Early stopping, best iteration is:
[304]	training's multi_logloss: 0.452915	valid_1's multi_logloss: 0.732126
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.615675	valid_1's multi_logloss: 0.752087
[200]	training's multi_logloss: 0.523052	valid_1's multi_logloss: 0.738353
Early stopping, best iteration is:
[245]	training's multi_logloss: 0.4897

0.7317003641934843

In [32]:
### For Submission
submission = pd.read_csv(data_path+'sample_submission.csv')
submission.iloc[:,1:] =  sub
submission.to_csv("_sub_10.csv", index = False)

## (2) RandomForest 

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

In [122]:
# StratifiedKFold
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

outcomes = []
sub = np.zeros((test.shape[0],3))
for n_fold, (train_index, val_index)  in enumerate(folds.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    rdf = RandomForestClassifier(random_state=2)
    rdf.fit(X_train, y_train)
    predictions = rdf.predict_proba(X_val)
    y_val_onehot = pd.get_dummies(y_val)

    sub += rdf.predict_proba(test)

    logloss = log_loss(y_val_onehot, predictions)
    outcomes.append(logloss)

sub = sub / 10
   
np.mean(outcomes)

0.6971342338475238

In [124]:
# for submission
submission = pd.read_csv(data_path+'sample_submission.csv')
submission.iloc[:,1:] =  sub
submission.to_csv("_sub_11.csv", index = False)

## (3) SVC

In [37]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV

### 선형 분리모델

In [31]:
# # cross_val_score
# svc_linear = SVC(kernel='linear')
# scores = cross_val_score(svc_linear, X, y, cv=10)
# print('검증 결과 점수:', scores)
# print('검증 결과 점수 평균:', scores.mean())

검증 결과 점수: [0.64134543 0.64134543 0.64134543 0.64134543 0.64134543 0.6409675
 0.6409675  0.6415879  0.6415879  0.6415879 ]
검증 결과 점수 평균: 0.6413425836623243


In [ ]:
# # GridSesarchCV
# param_grid = {'C': [0.01, 0.1, 1, 10, 25, 50]}
# svc_grid = GridSearchCV(svc_linear, param_grid=param_grid, cv=10)
# svc_grid.fit(X_train, y_train)
# y_proba = svc_grid.predict_proba(X_val)
# log_loss(y_val_onehot, y_proba)

In [ ]:
# # for submission
# submission = pd.read_csv(data_path+'sample_submission.csv')
# submission.iloc[:,1:] = svc_grid.predict_proba(test)
# submission.to_csv("_sub_11.csv", index=False)
# submission.head()

### 비선형 분리모델

In [ ]:
# # cross_val_score
# svc_rbf = SVC(kernel='rbf', gamma='auto')
# scores = cross_val_score(svc_rbf, X, y, cv=10)
# print('검증 결과 점수:', scores)
# print('검증 결과 점수 평균:', scores.mean())

In [ ]:
# # GridSesarchCV
# param_grid = {'C': [0.01, 0.1, 1, 10, 25, 50]}
# svc_grid = GridSearchCV(svc_rbf, param_grid=param_grid, cv=10)
# svc_grid.fit(X_train, y_train)
# y_proba = svc_grid.predict_proba(X_val)
# log_loss(y_val_onehot, y_proba)

In [ ]:
# # for submission
# submission = pd.read_csv(data_path+'sample_submission.csv')
# submission.iloc[:,1:] = svc_grid.predict_proba(test)
# submission.to_csv("_sub_12.csv", index=False)
# submission.head()

## (4) Naive Bayes

In [31]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import log_loss

In [34]:
nb = GaussianNB()
scores = cross_val_score(nb, X, y, cv=10, n_jobs=1, scoring='accuracy')
print('검증 결과 점수:', scores)
print('검증 결과 점수 평균:', scores.mean())

검증 결과 점수: [0.14814815 0.143613   0.1579743  0.15117158 0.15759637 0.15230537
 0.14814815 0.16408318 0.17240076 0.1731569 ]
검증 결과 점수 평균: 0.15685977478578073


In [33]:
# StratifiedKFold
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

outcomes = []
sub = np.zeros((test.shape[0],3))
for n_fold, (train_index, val_index)  in enumerate(folds.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    predictions = nb.predict_proba(X_val)
    y_val_onehot = pd.get_dummies(y_val)

    sub += nb.predict_proba(test)

    logloss = log_loss(y_val_onehot, predictions)
    outcomes.append(logloss)

sub = sub / 10
   
np.mean(outcomes)

5.362209091016423

## (5) K-NN

In [34]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import log_loss

In [28]:
knn = KNeighborsClassifier(weights='distance', metric='euclidean')
param_grid = [{'n_neighbors': range(3,12)}]
knn_grid = GridSearchCV(estimator=knn, param_grid=param_grid, cv=10, n_jobs=-1)
knn_grid.fit(X_train, y_train)

print('best hyper parameter:', knn_grid.best_params_)
print('best score: {0:.4f}'.format(knn_grid.best_score_))

best hyper parameter: {'n_neighbors': 11}
best score: 0.6529


In [29]:
# 예측확률 
y_proba = knn_grid.predict_proba(X_val)
y_proba

array([[0.07170638, 0.27128848, 0.65700514],
       [0.05722139, 0.43234449, 0.51043411],
       [0.0180889 , 0.08559089, 0.89632021],
       ...,
       [0.07799366, 0.61577847, 0.30622787],
       [0.13926941, 0.        , 0.86073059],
       [0.27841044, 0.26007274, 0.46151682]])

In [30]:
# 예측값 
y_val_onehot = pd.get_dummies(y_val)
y_val_onehot.head()

,credit
12594,1.0
16337,2.0
2178,2.0
5422,2.0
6711,2.0


In [31]:
# Logloss 
from sklearn.metrics import log_loss
log_loss(y_val_onehot, y_proba)

3.2384303114445245

In [35]:
# StratifiedKFold
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

outcomes = []
sub = np.zeros((test.shape[0],3))
for n_fold, (train_index, val_index)  in enumerate(folds.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    knn = KNeighborsClassifier(weights='distance', metric='euclidean')
    knn.fit(X_train, y_train)
    predictions = knn.predict_proba(X_val)
    y_val_onehot = pd.get_dummies(y_val)

    sub += knn.predict_proba(test)

    logloss = log_loss(y_val_onehot, predictions)
    outcomes.append(logloss)

sub = sub / 10
   
np.mean(outcomes)

5.060903381964775

In [36]:
# for submission
submission = pd.read_csv(data_path+'sample_submission.csv')
submission.iloc[:,1:] =  sub
submission.to_csv("_sub_02.csv", index = False)

## (6) Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

### Lasso

In [30]:
# StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
outcomes = []
sub = np.zeros((test.shape[0],3))
for n_fold, (train_index, val_index)  in enumerate(folds.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    lasso = LogisticRegression(penalty='l1', solver='liblinear', C=1, random_state=2)   
    lasso.fit(X_train, y_train)
    predictions = lasso.predict_proba(X_val)
    y_val_onehot = pd.get_dummies(y_val)

    sub += lasso.predict_proba(test)

    logloss = log_loss(y_val_onehot, predictions)
    outcomes.append(logloss)

sub = sub / 10
   
np.mean(outcomes)

0.8621577116682703

In [94]:
# for submission
submission = pd.read_csv(data_path+'sample_submission.csv')
submission.iloc[:,1:] =  sub
submission.to_csv("_sub_12.csv", index = False)

### Ridge

In [36]:
# StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
outcomes = []
sub = np.zeros((test.shape[0],3))
for n_fold, (train_index, val_index)  in enumerate(folds.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    ridge = LogisticRegression(penalty='l2', C=2, random_state=2)   
    ridge.fit(X_train, y_train)
    predictions = ridge.predict_proba(X_val)
    y_val_onehot = pd.get_dummies(y_val)

    sub += ridge.predict_proba(test)

    logloss = log_loss(y_val_onehot, predictions)
    outcomes.append(logloss)

sub = sub / 10
   
np.mean(outcomes)

0.8628186434830563

In [ ]:
# for submission
submission = pd.read_csv(data_path+'sample_submission.csv')
submission.iloc[:,1:] =  sub
submission.to_csv("_sub_13.csv", index = False)

## (7) XGBoost

In [57]:
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

In [58]:
# StratifiedKFold
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

outcomes = []
sub = np.zeros((test.shape[0],3))
for n_fold, (train_index, val_index)  in enumerate(folds.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    xgb = XGBClassifier(n_estimators=1000, random_state=2)
    xgb.fit(X_train, y_train,
           eval_set=[(X_train,y_train),(X_val,y_val)],
           early_stopping_rounds=30,
           verbose=100)
    predictions = xgb.predict_proba(X_val)
    y_val_onehot = pd.get_dummies(y_val)

    sub += xgb.predict_proba(test)

    logloss = log_loss(y_val_onehot, predictions)
    outcomes.append(logloss)

sub = sub / 10
   
np.mean(outcomes)

[0]	validation_0-merror:0.574422	validation_1-merror:0.57965
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 30 rounds.
[100]	validation_0-merror:0.528605	validation_1-merror:0.539972
[200]	validation_0-merror:0.485299	validation_1-merror:0.507759
[300]	validation_0-merror:0.457992	validation_1-merror:0.480849
[400]	validation_0-merror:0.439591	validation_1-merror:0.463367
[500]	validation_0-merror:0.423264	validation_1-merror:0.449813
[600]	validation_0-merror:0.408334	validation_1-merror:0.436457
Stopping. Best iteration:
[631]	validation_0-merror:0.402244	validation_1-merror:0.430367

[0]	validation_0-merror:0.576277	validation_1-merror:0.577097
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 30 rounds.
[100]	validation_0-merror:0.52782	validation_1-merror:0.529366
[200]	validat

0.9091072195162531

In [59]:
# for Submission
submission = pd.read_csv(data_path+'sample_submission.csv')
submission.iloc[:,1:] =  sub
submission.to_csv("_sub_14.csv", index = False)

## (8) 스태킹 앙상블

In [52]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

In [53]:
# 개별 모델 생성
knn_clf  = KNeighborsClassifier(n_neighbors=8)
rf_clf = RandomForestClassifier(n_estimators=500, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=500)
# lr_clf = LogisticRegression(C=10)
# xgb_clf = XGBClassifier(n_estimators=1000, random_state=2)

# 최종 Stacking 모델 생성
# lgbm = LGBMClassifier(n_estimators=1000, num_leaves=64, sumsample=0.8,
#                      min_child_samples=60, max_depth=128, random_state=2)
lr_final = LogisticRegression(C=10)

In [47]:
# # 개별 모델들을 학습
# #knn_clf.fit(X_train, y_train)
# rf_clf.fit(X_train , y_train)
# #dt_clf.fit(X_train , y_train)
# ada_clf.fit(X_train, y_train)
# lr_clf.fit(X_train, y_train)
# xgb_clf.fit(X_train, y_train,
#            eval_set=[(X_train,y_train),(X_val,y_val)],
#            early_stopping_rounds=30,
#            verbose=100)

# # 개별 모델 예측
# #knn_pred = knn_clf.predict_proba(X_val)
# rf_pred = rf_clf.predict_proba(X_val)
# #dt_pred = dt_clf.predict_proba(X_val)
# ada_pred = ada_clf.predict_proba(X_val)
# lr_pred = lr_clf.predict_proba(X_val)
# xgb_pred = xgb_clf.predict_proba(X_val)

# # 개별 모델 Logloss
# y_val_onehot = pd.get_dummies(y_val)
# print('KNN Logloss: {0:.4f}'.format(log_loss(y_val_onehot, knn_pred)))
# print('랜덤 포레스트 Logloss: {0:.4f}'.format(log_loss(y_val_onehot, rf_pred)))
# print('결정 트리 Logloss: {0:.4f}'.format(log_loss(y_val_onehot, dt_pred)))
# print('에이다부스트 Logloss: {0:.4f}'.format(log_loss(y_val_onehot, ada_pred)))
# print('로지스틱 회귀 Logloss: {0:.4f}'.format(log_loss(y_val_onehot, lr_pred)))
# print('XGBoost Logloss: {0:.4f}'.format(log_loss(y_val_onehot, xgb_pred)))

[0]	validation_0-merror:0.309844	validation_1-merror:0.300945
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 30 rounds.
Stopping. Best iteration:
[2]	validation_0-merror:0.309046	validation_1-merror:0.299811

랜덤 포레스트 Logloss: 0.8203
에이다부스트 Logloss: 1.0965
로지스틱 회귀 Logloss: 0.8543
XGBoost Logloss: 0.9850


In [55]:
# 개별 모델들을 학습
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train , y_train)
dt_clf.fit(X_train , y_train)
ada_clf.fit(X_train, y_train)

# 학습된 개별 모델들이 각자 반환하는 예측 데이터 셋을 생성하고 개별 모델의 정확도 측정 
knn_pred = knn_clf.predict(X_val)
rf_pred = rf_clf.predict(X_val)
dt_pred = dt_clf.predict(X_val)
ada_pred = ada_clf.predict(X_val)

print('KNN 정확도: {0:.4f}'.format(accuracy_score(y_val, knn_pred)))
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(y_val, rf_pred)))
print('결정 트리 정확도: {0:.4f}'.format(accuracy_score(y_val, dt_pred)))
print('에이다부스트 정확도: {0:.4f}'.format(accuracy_score(y_val, ada_pred)))

KNN 정확도: 0.6208
랜덤 포레스트 정확도: 0.6904
결정 트리 정확도: 0.6314
에이다부스트 정확도: 0.6994


In [56]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])

# transpose를 이용해 행과 열의 위치 교환. 컬럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦
pred = np.transpose(pred)

lr_final.fit(pred, y_val)
final = lr_final.predict(pred)
print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_val , final)))

최종 메타 모델의 예측 정확도: 0.7032


In [57]:
# step1
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정된 n_folds값으로 KFold 생성
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))

    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  

        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장
        test_pred[:, folder_counter] = model.predict(X_test_n)
   
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    

    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [58]:
# CV스태킹 알고리즘을 각 모델에 적용하여 메타 모델이 추후에 사용할 학습/테스트 데이터 만들기
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_val, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_val, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_val,  7)    
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_val, 7)

KeyError: "None of [Int64Index([ 3402,  3403,  3404,  3405,  3406,  3407,  3408,  3409,  3410,\n             3411,\n            ...\n            23802, 23803, 23804, 23805, 23806, 23807, 23808, 23809, 23810,\n            23811],\n           dtype='int64', length=20410)] are in the [columns]"

In [59]:
# step2: 생성된 개별 모델들의 학습/테스트 데이터 합쳐주기
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print('원본 학습 피처 데이터 Shape:',X_train.shape, '원본 테스트 피처 Shape:',X_test.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape, '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

NameError: name 'knn_train' is not defined

In [ ]:
# 최종 메타 모델 학습/예측
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, stack_final)))